# Notebook

This notebook is containing 1st version of script, it serves as a workspace.

## TODO:
* #odejmij wartość 1. bodźcda czasu + podziel przez 10000, aby mieć milisekundy DONE
* #wartośc block end-block start DONE
* #targety - regressor of no interest DONE
* #możesz zbierać fragmenty w których pojawia się cue
* #podsumowania behawioralne
* #Usprawnić kod



In [1]:
import pandas as pd
import numpy as np
import os
from functools import reduce

In [74]:
def read_file(filename):
    """
    Read file, check whether it contains proper
    columns (based on first one) and return 
    DataFrame.
    """
    file = pd.read_csv(filename,"\t",skiprows=2)
    if np.any(file.columns.str.contains('Subject')):
        return file
    else:
        file = pd.read_csv(filename,"\t",skiprows=3)
        file = file.dropna(subset = ['Subject'])
        return file

def time_correction(data):
    """
    Corrects time in data from Presentation. 
    Reduce all Time column by time value of first event, 
    then divides by 10000 (to turn into ms).
    Returns whole DataFrame.
    """
    data['Time'] = (data['Time'] - data['Time'].iloc[0])/10000
    return data

def get_arg_block(data):
    """Get arg of blocks in data"""
    #get condition (all rows with word block in Code col)
    cond = ['block' in n.lower() for n in data['Code']]
    return (data
            .loc[cond]
            .index)

def get_list_endpoints(arg_data_part):
    """ 
    Get arg of starting and ending block, returns 
    list with shape [[arg_start,arg_end]...]
    """
    return [[arg_data_part[arg],arg_data_part[arg+1]] for \
            arg in range(0,len(arg_data_part),2)]

def present_target(data,start_time,end_time):
    """
    Check for the presence of the target.
    """
    return np.any((data
                   .loc[start_time:end_time,'Code']
                   .str.contains('target')))

def get_time_point_results(data,time_points):
    """
    Get results from data, for a given time_point.
    Return a dictionary with colnames linked to values
    """
    #get code event
    code_name = data.loc[time_points[0],'Code']
    #get starting point of event
    start_time = data.loc[time_points[0],'Time']
    #calculate enging point of event
    end_time = (data.loc[time_points[1],'Time']-start_time)
    #check the presence of a target in event
    target_val = present_target(data,
                                time_points[0],
                                time_points[1])
    #return a tuple with all
    return {'Code':code_name,
            'StartTime':start_time,
            'EndTime':end_time,
            'target':target_val}

def merge_blocks(previous_block,endpoints):
    """
    Merge previous block with new one.
    """
    tmp_series = pd.Series(endpoints)
    try:
        return previous_block.append(tmp_series,
                                     ignore_index=True)
    except:
        return pd.DataFrame().append(tmp_series,
                                     ignore_index=True)

def get_block_result(data):
    """
    Pipeline for obtaining block results.
    """
    #Get arguments of blocks
    blocks_arg = get_arg_block(data)
    #Get endpoints for each block
    endpoints = get_list_endpoints(blocks_arg)
    #Fix for first iteration in map
    endpoints.insert(0,[0,0])
    #Get iterator for each time block
    time_block_result = map(lambda x : get_time_point_results(data,x),
                            endpoints)
    #Get result
    return reduce(merge_blocks,time_block_result)

def get_cue_results(data,placeholder=pd.DataFrame()):
    args = (data
            [data['Code'].str.contains('cue')]
            .index)
    placeholder['Code'] = data.loc[args,'Code']
    placeholder['EndTime'] = (data.loc[args+1]['Time'].values -\
                              data.loc[args]['Time'])
    placeholder['StartTime'] = data.loc[args,'Time']
    return placeholder

#Read file
data = read_file("6_runs_SIGHTED-6_runs_Fastloc_1back_sighted_run1.log")
#Perform time correction
data = time_correction(data)
#get block results
result = get_block_result(data)
#Append cue results, fill Nan with 0
#and reset index, for aesthethical purposes
result = (result
          .append(get_cue_results(data))
          .fillna(0)
          .reset_index(drop=True))
result

,Code,EndTime,StartTime,target
0,BLOCK_START_VIS_CONTROL,8.0669,7.5545,0.0
1,BLOCK_START_VIS_WORDS,8.0669,18.6550,0.0
2,BLOCK_START_VIS_PSEUDOWORDS,8.0670,31.7556,1.0
3,BLOCK_START_AUDIO_CONTROL,8.0003,45.3896,0.0
4,BLOCK_START_AUDIO_PSEUDOWORDS,8.0003,57.4235,1.0
5,BLOCK_START_AUDIO_WORDS,8.0003,68.9574,0.0
6,BLOCK_START_VIS_CONTROL,8.0669,85.0249,1.0
7,BLOCK_START_VIS_WORDS,8.0669,95.6254,0.0
8,BLOCK_START_VIS_PSEUDOWORDS,8.0669,107.7260,1.0
9,BLOCK_START_AUDIO_WORDS,8.0003,122.3600,1.0


,Code,EndTime,StartTime
2,cue_open_eyes,3.0154,2.0222
25,cue_close_eyes,1.0166,41.8562
48,cue_open_eyes,3.0167,79.4913
71,cue_close_eyes,1.0167,118.8265
94,cue_close_eyes,1.0166,156.9617
117,cue_open_eyes,3.0167,195.0969
